# Module 10: Building with AI Agents

**Goal:** Understand when and how to build AI agents that take actions

**Time:** ~25 minutes

**What you'll do:**
1. Understand the spectrum: chatbot → agent
2. Design tool-calling patterns
3. Implement guardrails and stop conditions
4. Know when NOT to use agents

---

## Part 1: What Is an Agent?

An agent doesn't just generate text-it takes ACTIONS.

In [ ]:
spectrum = """
┌─────────────────────────────────────────────────────────────────────────────┐
│                    THE AUTOMATION SPECTRUM                                  │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  CHATBOT          ASSISTANT         WORKFLOW           AGENT               │
│  ───────          ─────────         ────────           ─────               │
│                                                                             │
│  • Static         • Retrieves       • Pre-defined      • Plans &           │
│    responses        information       steps              decides           │
│  • No actions     • Single tool     • Multi-step       • Uses tools        │
│  • Scripted       • Human-guided    • Automated        • Autonomous        │
│                                                                             │
│  Risk: Low ───────────────────────────────────────────────────▶ Risk: High │
│  Control: Full ───────────────────────────────────────────▶ Control: Less  │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘

KEY INSIGHT: Start left, move right only when needed.
"""
print(spectrum)

In [ ]:
# Examples of each level
examples = """
=== Examples ===

CHATBOT (no actions):
  "What are your store hours?" → Static FAQ response

ASSISTANT (reads data):
  "What's my order status?" → Looks up order in database, returns info

WORKFLOW (predefined steps):
  "Process this refund" → IF valid_order AND within_policy THEN issue_refund

AGENT (plans and decides):
  "Help me with my billing issue" → Reads account, identifies problem,
  decides whether to offer credit OR schedule callback OR escalate
"""
print(examples)

---

## Part 2: Tool Calling

Agents work by calling tools. You define what tools are available.

In [ ]:
# Define tools for a support agent
tools = [
    {
        "name": "lookup_customer",
        "description": "Get customer account information by email or customer ID",
        "parameters": {"customer_id": "string"},
        "risk": "low"  # Read-only
    },
    {
        "name": "get_order_status",
        "description": "Check the status of a specific order",
        "parameters": {"order_id": "string"},
        "risk": "low"  # Read-only
    },
    {
        "name": "issue_refund",
        "description": "Issue a refund for an order",
        "parameters": {"order_id": "string", "amount": "float", "reason": "string"},
        "risk": "high"  # Financial action
    },
    {
        "name": "send_email",
        "description": "Send an email to the customer",
        "parameters": {"to": "string", "subject": "string", "body": "string"},
        "risk": "medium"  # External communication
    },
    {
        "name": "escalate_to_human",
        "description": "Transfer the conversation to a human agent",
        "parameters": {"reason": "string", "priority": "string"},
        "risk": "low"  # Safe fallback
    }
]

print("=== Available Tools ===")
for tool in tools:
    risk_emoji = {"low": "🟢", "medium": "🟡", "high": "🔴"}[tool['risk']]
    print(f"{risk_emoji} {tool['name']}: {tool['description']}")

In [ ]:
# Simulated tool implementations

def lookup_customer(customer_id):
    """Simulated customer lookup."""
    return {
        "customer_id": customer_id,
        "name": "Jane Smith",
        "email": "jane@example.com",
        "plan": "premium",
        "tenure_months": 14,
        "open_orders": 1
    }

def get_order_status(order_id):
    """Simulated order lookup."""
    return {
        "order_id": order_id,
        "status": "shipped",
        "tracking": "1Z999AA10123456784",
        "estimated_delivery": "2024-01-20",
        "amount": 49.99
    }

def issue_refund(order_id, amount, reason):
    """Simulated refund (would need approval in real system)."""
    return {
        "status": "pending_approval",
        "refund_id": "REF-123456",
        "amount": amount,
        "reason": reason
    }

# Tool router
tool_map = {
    "lookup_customer": lookup_customer,
    "get_order_status": get_order_status,
    "issue_refund": issue_refund
}

---

## Part 3: Agent Loop

The basic pattern: Think → Act → Observe → Repeat

In [ ]:
agent_loop = """
┌─────────────────────────────────────────────────────────────────────────────┐
│                        THE AGENT LOOP                                       │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│   ┌──────────────────────────────────────────────────────────────┐         │
│   │                                                              │         │
│   ▼                                                              │         │
│  THINK              ACT               OBSERVE                    │         │
│  ─────              ───               ───────                    │         │
│  What do I          Call a tool       What happened?             │         │
│  need to do?        or respond        Did it work?               │         │
│                                                                  │         │
│   │                  │                  │                        │         │
│   └──────────────────┴──────────────────┴────────────────────────┘         │
│                                                                             │
│  STOP CONDITIONS:                                                           │
│   • Task completed                                                          │
│   • Max steps reached                                                       │
│   • Error or uncertainty                                                    │
│   • Human escalation needed                                                 │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
"""
print(agent_loop)

In [ ]:
# Simplified agent execution trace

def simulate_agent(user_request):
    """Show what an agent would do step-by-step."""
    print(f"USER: {user_request}")
    print("\n--- Agent Execution ---")
    
    # Step 1: Think
    print("\n[THINK] Need to check the order status for this customer.")
    
    # Step 2: Act
    print("[ACT] Calling: lookup_customer(customer_id='C-12345')")
    customer = lookup_customer('C-12345')
    
    # Step 3: Observe
    print(f"[OBSERVE] Found customer: {customer['name']}, {customer['open_orders']} open orders")
    
    # Step 4: Think again
    print("\n[THINK] Customer has an open order. Let me check its status.")
    
    # Step 5: Act again
    print("[ACT] Calling: get_order_status(order_id='ORD-98765')")
    order = get_order_status('ORD-98765')
    
    # Step 6: Observe
    print(f"[OBSERVE] Order is {order['status']}, ETA: {order['estimated_delivery']}")
    
    # Step 7: Respond
    print("\n[RESPOND] Generating response to user...")
    response = f"""Hi Jane! Your order ORD-98765 has been shipped and is on its way. 
Tracking number: {order['tracking']}
Estimated delivery: {order['estimated_delivery']}

Is there anything else I can help you with?"""
    
    print(f"\nAGENT: {response}")

simulate_agent("Where is my package?")

---

## Part 4: Guardrails

Critical: What can the agent NOT do?

In [ ]:
guardrails = """
┌─────────────────────────────────────────────────────────────────────────────┐
│  GUARDRAIL TYPE        │  IMPLEMENTATION                                   │
├─────────────────────────────────────────────────────────────────────────────┤
│  Action limits         │  Max refund amount ($50 without approval)         │
│                        │  Max emails per conversation (2)                  │
│                        │  No account deletion                              │
├─────────────────────────────────────────────────────────────────────────────┤
│  Content filters       │  No PII in logs                                   │
│                        │  No competitor mentions                           │
│                        │  Approved response templates for legal topics     │
├─────────────────────────────────────────────────────────────────────────────┤
│  Escalation triggers   │  Customer uses angry keywords                     │
│                        │  Request beyond agent authority                   │
│                        │  Agent is uncertain (confidence < 0.7)            │
├─────────────────────────────────────────────────────────────────────────────┤
│  Loop prevention       │  Max 5 tool calls per request                     │
│                        │  No repeated calls with same parameters           │
│                        │  Timeout after 30 seconds                         │
└─────────────────────────────────────────────────────────────────────────────┘
"""
print(guardrails)

In [ ]:
# Implement basic guardrails

class Guardrails:
    MAX_REFUND = 50.0
    MAX_STEPS = 5
    ESCALATION_KEYWORDS = ['lawsuit', 'lawyer', 'sue', 'legal', 'angry', 'furious']
    
    @staticmethod
    def check_refund(amount):
        if amount > Guardrails.MAX_REFUND:
            return False, f"Refund ${amount} exceeds limit ${Guardrails.MAX_REFUND}. Needs human approval."
        return True, "OK"
    
    @staticmethod
    def check_escalation(text):
        text_lower = text.lower()
        for keyword in Guardrails.ESCALATION_KEYWORDS:
            if keyword in text_lower:
                return True, f"Escalation triggered by keyword: '{keyword}'"
        return False, "OK"

# Test guardrails
print("=== Guardrail Tests ===")

# Refund check
ok, msg = Guardrails.check_refund(25.0)
print(f"Refund $25: {'✓' if ok else '✗'} {msg}")

ok, msg = Guardrails.check_refund(100.0)
print(f"Refund $100: {'✓' if ok else '✗'} {msg}")

# Escalation check
escalate, msg = Guardrails.check_escalation("I'm frustrated with the service")
print(f"\n'frustrated': Escalate={escalate}")

escalate, msg = Guardrails.check_escalation("I will contact my lawyer!")
print(f"'lawyer': Escalate={escalate} - {msg}")

---

## Part 5: When NOT to Use Agents

In [ ]:
when_not_to_use = """
┌─────────────────────────────────────────────────────────────────────────────┐
│  🚫 DON'T USE AN AGENT WHEN...                                             │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  1. DETERMINISTIC FLOW WORKS                                                │
│     If you can write if/else logic, do that instead.                       │
│     Agents add uncertainty-only use when you need flexibility.             │
│                                                                             │
│  2. STAKES ARE TOO HIGH                                                     │
│     Medical diagnosis, legal advice, financial transactions > $X           │
│     Human review required for consequential decisions.                     │
│                                                                             │
│  3. LATENCY MATTERS                                                         │
│     Each LLM call = 500ms-2s. Multi-step agents are slow.                  │
│     Use traditional ML for real-time decisions.                            │
│                                                                             │
│  4. AUDITABILITY REQUIRED                                                   │
│     "Why did the system do X?" is hard to answer with agents.              │
│     Regulatory requirements may need deterministic logic.                  │
│                                                                             │
│  5. YOU CAN'T DEFINE GOOD GUARDRAILS                                        │
│     If you don't know what "wrong" looks like, don't automate.             │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
"""
print(when_not_to_use)

In [ ]:
# TODO: For each scenario, decide: Agent? Workflow? Or traditional code?

scenarios = [
    "Route support tickets to the right department",
    "Approve/deny credit card applications",
    "Book a meeting based on email conversation",
    "Calculate shipping cost based on weight and destination",
    "Answer 'How do I reset my password?' questions"
]

# Your answers (Agent / Workflow / Traditional Code):
answers = [
    "???",  # Ticket routing
    "???",  # Credit approval
    "???",  # Meeting booking
    "???",  # Shipping cost
    "???"   # Password reset
]

print("=== Build vs. Agent Decision ===")
for s, a in zip(scenarios, answers):
    print(f"• {s}")
    print(f"  → {a}\n")

---

## Part 6: Production Checklist

In [ ]:
production_checklist = """
=== Agent Production Checklist ===

BEFORE LAUNCH:
□ Define all available tools and their risk levels
□ Implement guardrails for each high-risk action
□ Set up human escalation path
□ Add logging for all tool calls and decisions
□ Set max steps and timeout limits
□ Test with adversarial inputs ("ignore instructions and...")

MONITORING:
□ Track success rate (task completed without escalation)
□ Track average steps per request
□ Track latency (P50, P95, P99)
□ Track cost per request
□ Flag and review escalations
□ Sample random conversations for quality

ITERATION:
□ Analyze failure modes weekly
□ Update guardrails based on edge cases
□ Retrain/update prompts based on feedback
□ Expand tool access gradually
"""
print(production_checklist)

---

## 📝 Final Exercise: Explain It

Your CEO sees a demo of an AI agent and asks: "Why don't we just have an agent handle all customer support?"

Write a 5-6 sentence response covering capability, risk, and recommended scope.

In [ ]:
# Write your response:

ceo_response = """
YOUR RESPONSE HERE
"""

print(ceo_response)

---

## ✅ Module 10 Complete!

**What you learned:**
- The spectrum from chatbot to autonomous agent
- How tool calling works
- Essential guardrails for production agents
- When NOT to use agents

In [ ]:
# Summary
print("=== Module 10 Summary ===")
print("\nKey takeaways:")
print("  • Agents = LLMs + Tools + Loop")
print("  • Start with workflows, add agents only when needed")
print("  • Guardrails are not optional")
print("  • Monitor, measure, iterate")

---

# 🎉 Course Complete!

You've covered the full ML journey:

1. **Problem Framing** - Is this even an ML problem?
2. **Logistic Regression** - Your first model
3. **Decision Trees** - When linear isn't enough
4. **Gradient Boosting** - Production-grade models
5. **Feature Engineering** - The art of creating signals
6. **Evaluation** - Measuring what matters
7. **Clustering** - Finding natural groups
8. **Embeddings** - Turning text into numbers
9. **Transformers** - Modern language models
10. **Agents** - LLMs that take actions

**What's next?**
- Apply these concepts to your own projects
- Build the capstone project
- Share what you've learned